# Part 1: Heterogeneous treatment effects using causal trees and forests

For this part, we will be using experimental data for computing heterogeneous effects through causal trees and forests. For all exercises, the predictors X are all variables that are not the outcome Y or the treatment D.

1.4. Heterogeneous effects with causal forests (3 points). Use causal forests like we saw in class. For Python, you should use the econml package; for R, use the grf package; and for Julia, you will need to use the auxiliary variable Y∗ computed in the previous exercise and fit a random forest regressor. Report the importance of the prediction variables.

1.5. Plot heterogeneous effects (1.5 points). Plot how the predicted treatment effect changes depending on a variable of your choice. (You can see the last example in PD11 for clarification of what you should do in this exercise)

1.1. Load the data (1 points). This is data for and experiment regarding the National Supported Work Demonstration (NSW) job-training program. You can find the data here, and read a description of the data here. For further details of the experiment and the program, you can use this link

In [14]:
using CSV
using DataFrames  

url = "https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/Labs/Assignment/Assignment_5/data/experimental/experimental_control.csv"
df = CSV.read(download(url), DataFrame)

first(df, 5)

Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,1,37,11,1,0,1,1,0.0,0.0,9930.05
2,1,22,9,0,1,0,1,0.0,0.0,3595.89
3,1,30,12,1,0,0,0,0.0,0.0,24909.4
4,1,27,11,1,0,0,1,0.0,0.0,7506.15
5,1,33,8,1,0,0,1,0.0,0.0,289.79


1.2. Find the ATE (1.5 points). With re78 as the outcome variable of interest, find the Average Treatment Effect of participation in the program. Specifically, you should find it by calculating the difference between the means of the treatment group and the control group (the Simple Difference of Means or SDM). What can you say about the program?

Con re78 como la variable de resultado de interés, calcula el Efecto Promedio del Tratamiento (Average Treatment Effect, ATE) de la participación en el programa. Específicamente, deberías encontrarlo calculando la diferencia entre las medias del grupo de tratamiento y del grupo de control (la Diferencia Simple de Medias o SDM). ¿Qué puedes decir sobre el programa?

In [20]:
using Statistics 

mean_treat = mean(df[df.treat .== 1, :re78])
mean_control = mean(df[df.treat .== 0, :re78])

# Calcular el ATE
ATE = mean_treat - mean_control

println("El Average Treatment Effect (ATE) es: $ATE")

El Average Treatment Effect (ATE) es: 1794.3423818501024


The program had a positive impact on the participants' income, increasing their earnings by an average of 1794.34 monetary units compared to the control group. It can be concluded that the program is effective in improving participants' income.

1.3. Heterogeneous effects with causal trees (3 points). Use causal trees like we saw in class. For Python, you should use the econml package; for R, use the grf package; and for Julia, you will need to create the auxiliary variable 
Y∗ and fit a decision tree regressor. Report the splits the tree finds and interpret them.

Utiliza árboles causales como se explicó en clase. Para Python, deberías usar el paquete econml; para R, utiliza el paquete grf; y para Julia, necesitarás crear la variable auxiliar 𝑌∗Y ∗  y ajustar un árbol de decisión regresor. Reporta las divisiones que los árboles encuentran e interpreta los resultados.

In [21]:
using DecisionTree

# Definir las variables predictoras y el resultado auxiliar
X = select(df, [:age, :educ])  # Predictores
Y = df.re78  # Variable de resultado

# Entrenar un árbol de decisión
tree_model = DecisionTreeRegressor(max_depth=3)
fit!(tree_model, Matrix(X), Y)

# Imprimir las divisiones del árbol
println("Divisiones del árbol causal:")
print_tree(tree_model)


ArgumentError: ArgumentError: Package DecisionTree not found in current path.
- Run `import Pkg; Pkg.add("DecisionTree")` to install the DecisionTree package.

Usa bosques causales como se explicó en clase. Para Python, deberías usar el paquete econml; para R, utiliza el paquete grf; y para Julia, necesitarás usar la variable auxiliar 𝑌∗Y ∗  calculada en el ejercicio anterior y ajustar un regresor aleatorio de bosques. Reporta la importancia de las variables de predicción.

Traza cómo cambia el efecto predicho del tratamiento dependiendo de una variable de tu elección. (Puedes ver el último ejemplo en PD11 para una aclaración de lo que debes hacer en este ejercicio).